In [5]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import evaluate  # Replaces load_metric
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

# Example usage of evaluate
metric = evaluate.load("accuracy")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mubashirmian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
pip install evaluate


Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

mps


In [11]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt) # passing same model to tokenizer

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device) # loading it into my gpy


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
!unzip summarizer-data.zip
     


Archive:  summarizer-data.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
   creating: samsum_dataset/
 extracting: samsum_dataset/dataset_dict.json  
   creating: samsum_dataset/test/
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
   creating: samsum_dataset/train/
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
   creating: samsum_dataset/validation/
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [9]:
dataset_samsum = load_from_disk("samsum_dataset")

In [10]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengthss: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialoge:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary")

print(dataset_samsum["test"][1]["summary"])


Split lengthss: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialoge:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary
Eric and Rob are going to watch a stand-up on youtube.


In [16]:

def convert_examples_to_feautes(example_batch):
    input_encodings = tokenizer(example_batch['dialouge'], max_length = 1024, truncation = True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'],
                                     max_lenght = 120, truncation = True)
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask' : input_encodings['attention_mask'],
        'labels' : target_encoding['input_ids']
    }